In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


import tensorflow as tf
from keras.models import Sequential
from keras import Input
from keras.losses import binary_crossentropy
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

In [54]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [55]:
state0=np.load('../Data/new/2gsps/state0.npy')
state1=np.load('../Data/new/2gsps/state1.npy')
output0=np.zeros((state0.shape[0]))
output1=np.ones((state1.shape[0]))
print(state0.shape)

(1000, 8192, 2)


In [56]:
x=np.vstack((state0,state1))
print(x.shape)
y=np.hstack((output0,output1))
print(y.shape)

(2000, 8192, 2)
(2000,)


In [60]:

st,ed=700,2700
x_train, x_test, y_train, y_test = train_test_split(x[:,st:ed,:], y, test_size=0.30, random_state=45)
print(x_train.shape)
print(y_train.sum())

x_train=np.mean(x_train,axis=1)
x_test=np.mean(x_test,axis=1)

# x_train=x_train.reshape((x_train.shape[0],x_train.shape[1]*2))
# x_test=x_test.reshape((x_test.shape[0],x_test.shape[1]*2))
print(x_train.shape)

(1400, 2000, 2)
704.0
(1400, 2)


## Logistic Regression

In [61]:
# x_trainC=x_train.reshape((x_train.shape[0],x_train.shape[1]*2))
# x_testC=x_test.reshape((x_test.shape[0],x_test.shape[1]*2))
classifier = LogisticRegression(random_state = 42, max_iter=500)
classifier.fit(x_train, y_train)
print(classifier.score(x_test,y_test))
pred = classifier.predict(x_test)

0.7833333333333333


## SVM

In [62]:
model=SVC(decision_function_shape='ovo')
model.fit(x_train,y_train)
print(model.score(x_test,y_test))

0.795


## GMM

In [46]:
Y=list(zip(np.append(np.mean(state0[:,st:ed,0],axis=1),np.mean(state1[:,st:ed,0],axis=1)),np.append(np.mean(state0[:,st:ed,1],axis=1),np.mean(state1[:,st:ed,1],axis=1))))
gmm_Y=GaussianMixture(n_components=2,covariance_type='full').fit(Y)
bitstring=np.split(gmm_Y.predict(Y),2)
print('P(0|0)=',len(bitstring[0][bitstring[0]==0])/len(bitstring[0]))
print('P(1|1)=',len(bitstring[1][bitstring[1]==1])/len(bitstring[1]))
X=list(zip(np.mean(state0[:,st:ed,0],axis=1),np.mean(state0[:,st:ed,1],axis=1)))
gmm_Y_1=GaussianMixture(n_components=1,covariance_type='spherical').fit(X)
X=list(zip(np.mean(state1[:,st:ed,0],axis=1),np.mean(state1[:,st:ed,1],axis=1)))
gmm_Y_2=GaussianMixture(n_components=1,covariance_type='spherical').fit(X)

P(0|0)= 0.758
P(1|1)= 0.82


## NN

In [26]:
def arch():
    model=Sequential()
    model.add(Input(shape=(2)))
    model.add(Dense(32,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(16,activation='relu'))
    model.add(Dropout(0.5))
    # model.add(Dense(16,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(16,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(32,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(16,activation='relu'))
    # # model.add(Dropout(0.8))
    # model.add(Dense(4,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    return model
arch().summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 32)                96        
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_7 (Dense)             (None, 16)                528       
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 17        
                                                                 
Total params: 641 (2.50 KB)
Trainable params: 641 (2.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
import keras.backend as K

def bceloss(y_true, y_pred):
    # Clip the prediction value to prevent log(0) error
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Calculate the weighted binary cross entropy loss
    loss = -(0.1 * (y_true * K.log(y_pred)) + 0.9*((1 - y_true) * K.log(1 - y_pred)))
    return K.mean(loss, axis=-1)

model=arch()
model.summary
opt=SGD( learning_rate=0.01, momentum=0.9)
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
model_path="../Model/NN2/{epoch:02d}-{val_accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [14]:
H=model.fit(x_train,y_train,
          validation_data=(x_test, y_test),
          epochs=100,batch_size=16,
          callbacks=callbacks_list,
          verbose=1)

Epoch 1/100
60/88 [===================>..........] - ETA: 0s - loss: 0.6729 - accuracy: 0.6177 
Epoch 1: val_accuracy improved from -inf to 0.90500, saving model to ../Model/NN2/01-0.9050.h5
88/88 [==============================] - 1s 3ms/step - loss: 0.6365 - accuracy: 0.6486 - val_loss: 0.3724 - val_accuracy: 0.9050
Epoch 2/100
67/88 [=====================>........] - ETA: 0s - loss: 0.4214 - accuracy: 0.7854
Epoch 2: val_accuracy improved from 0.90500 to 0.90667, saving model to ../Model/NN2/02-0.9067.h5
88/88 [==============================] - 0s 2ms/step - loss: 0.4078 - accuracy: 0.7993 - val_loss: 0.3105 - val_accuracy: 0.9067
Epoch 3/100
 1/88 [..............................] - ETA: 0s - loss: 0.4330 - accuracy: 0.7500

/global/homes/n/nrvora/.conda/envs/readout/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


67/88 [=====================>........] - ETA: 0s - loss: 0.3502 - accuracy: 0.8610
Epoch 3: val_accuracy improved from 0.90667 to 0.91000, saving model to ../Model/NN2/03-0.9100.h5
88/88 [==============================] - 0s 2ms/step - loss: 0.3445 - accuracy: 0.8664 - val_loss: 0.2936 - val_accuracy: 0.9100
Epoch 4/100
70/88 [======================>.......] - ETA: 0s - loss: 0.3020 - accuracy: 0.9089
Epoch 4: val_accuracy improved from 0.91000 to 0.91167, saving model to ../Model/NN2/04-0.9117.h5
88/88 [==============================] - 0s 2ms/step - loss: 0.3094 - accuracy: 0.9071 - val_loss: 0.2934 - val_accuracy: 0.9117
Epoch 5/100
71/88 [=======================>......] - ETA: 0s - loss: 0.2849 - accuracy: 0.9190
Epoch 5: val_accuracy did not improve from 0.91167
88/88 [==============================] - 0s 2ms/step - loss: 0.2838 - accuracy: 0.9164 - val_loss: 0.2985 - val_accuracy: 0.9117
Epoch 6/100
71/88 [=======================>......] - ETA: 0s - loss: 0.2675 - accuracy: 0.919

In [15]:
count=0
count1=0
y_pred=model.predict(x_test)
for i in range(y_pred.shape[0]):
    if y_test[i]==0:
        count+=1
        if y_pred[i,0]<0.5:
            count1+=1
print(count,count1)

19/19 [==============================] - 0s 586us/step
304 283


## GDA

In [47]:
import numpy as np
from scipy.stats import multivariate_normal

class GaussianDiscriminantAnalysis:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_priors = np.array([np.mean(y == c) for c in self.classes])

        self.means = []
        self.cov_matrices = []
        for c in self.classes:
            X_c = X[y == c]
            mean_c = np.mean(X_c, axis=0)
            cov_matrix_c = np.cov(X_c, rowvar=False)
            self.means.append(mean_c)
            self.cov_matrices.append(cov_matrix_c)

    def predict(self, X):
        posteriors = []

        for c in self.classes:
            class_prior = self.class_priors[int(c)]
            mean = self.means[int(c)]
            cov_matrix = self.cov_matrices[int(c)]
            mvn = multivariate_normal(mean=mean, cov=cov_matrix)
            posterior = class_prior * mvn.pdf(X)
            posteriors.append(posterior)

        posteriors = np.array(posteriors).T
        predicted_labels = np.argmax(posteriors, axis=1)

        return predicted_labels



# Initialize and train GDA
gda = GaussianDiscriminantAnalysis()
gda.fit(x_train, y_train)

# Make predictions
y_pred = gda.predict(x_train)
# Calculate accuracy
accuracy = np.mean(y_pred == y_train)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 79.00%
